## Train the neural network

In this notebook it is shown how to train the neural network on the simulated HRTEM images generated with make_training_data.py. The neural network is trained for many epochs and in each epoch the set of weights is saved in a dedicated folder. The training is performed on batches of images, and the model's parameters (weights and bias) are update using the gradient descent and backpropagation algorithm. 
The performance of the training is evaluated by computing the R2 score between the predicted and the true heights.

In [1]:
import numpy as np
import keras
from keras.models import Model
from keras.utils import multi_gpu_model
from net_architecture import CNN
from data_preparation import DataSet,DataEntry,load,get_data,Training_Data_Generator
from performance import performance_CH,get_performance_on_batch_train
import os
from datetime import datetime
import platform
import sys
import time
import shutil

Using TensorFlow backend.


Loading the model:

- input_channel=1: the HRTEM images are simulated with a single defocus value.
- input_shape=(256,256): image size.
- output channel=1: only a single output class is present in the regression scheme. In a standard semantic segmenatation procedure, the output channel is >1, corresponding to the number of classes to predict. In the classification model we have described in the manuscript and it is not present here, we have considered a total of 16 classes corresponding to column heights ranging from 1 to 16.

The model can be in serial or parallel mode depending on the gpus available on the user's machine.

In [3]:
input_channel=1
input_shape=(256,256)
input_tensor = keras.Input(shape=input_shape+(input_channel,))
output_channel=1
serial_model=CNN(input_tensor,output_channels=output_channel)

numgpus = 1

if numgpus >1:
    model=multi_gpu_model(serial_model,gpus=numgpus)
else:
    model=serial_model
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

Load the training data from folder. The folder 'data/training/' contains 2 subfolder: 'images' and 'labels' which contain the simulated HRTEM image and the corresponding labels generated with make_training_data.py.

In [4]:
training_data_path='data/training/'
training_data=load(training_data_path)

Define a generator of training batches. Here a batch size of 2 is adopted (the weights and the bias are update with a 2 images training), but this value can be changed depending on the user's needs. A batch training corresponds to a step of the epoch. The total number of steps in an epoch is equal to the number of training images divided by the batch size.

In [5]:
batch_size=2
training_data_generator=Training_Data_Generator(training_data,batch_size)
num_training_data=training_data.num_examples
steps_per_epoch=num_training_data//batch_size

Here a total number of 50 epoch is defined, but this value can be changed by the user. At each epoch the set of weights is saved in the folder 'weights/trained_weights/'. In this way, the weights can be loaded in test model to evaluate the performance of the neural network on unseen images.

In [6]:
num_epochs=50
total_num_steps=num_epochs*steps_per_epoch
weights_folder_path='weights/trained_weights/epoch-{}.h5'

Define the lists to save the metrics of the model for each epoch. The performance of the model is evaluated on each single image, then an average of all the images contained in a batch is calculated to obtain the performance in a single batch. Then, the average of the performance of all the batches is calculated to obtaine the performance in a single epoch. This value is saved in the dedicated list shown in the cell below. Here we consider the loss (mean squared error 'mse', the accuracy and R2 between the predicted and the true heights.

In [7]:
loss_all_epochs=[]
accuracy_all_epochs=[]
r2_heights_all_epochs=[]

In [ ]:
# Beginning of training

print('Training takes action')
before = time.time()

# for loop over the epochs
for epoch in range(num_epochs):
    
    # define the lists containg the performance in each training step in an epoch
    loss_in_steps=[]
    accuracy_in_steps=[]
    r2_heights_in_steps=[]
    
    # for loop over the training steps
    for i in range(steps_per_epoch):
        
        # lists corresponding to a batch of images and labels
        batch_images = []
        batch_labels = []
           
        # load the batch
        for b in range(batch_size):
            # generation of the training images and labels
            img, lbl = training_data_generator.next_example()
            
            # populating the images and labels batches
            batch_images.append(img)
            batch_labels.append(lbl)
        batch_images = np.concatenate(batch_images)
        batch_labels = np.concatenate(batch_labels)
        
        # training of the model in a batch of images
        performance_in_batch=model.train_on_batch(batch_images, batch_labels)
            
        # evaluating the model training on the batch
        loss_in_batch=performance_in_batch[0]
        accuracy_in_batch=performance_in_batch[1]
        
        # the R2 between the predicted and the true heights is calculated using the function
        # 'get_performance_on_batch_train' defined in the 'perforamance.py' file.
        r2_heights_in_batch=get_performance_on_batch_train(model,batch_images,batch_labels,batch_size)
            
        # print the step of the learning process
        print("Epoch: {}/{} Batch: {}/{}   [{}/{}]".format(epoch, num_epochs,
                                                                   i, steps_per_epoch,
                                                                   (i + epoch*steps_per_epoch)*batch_size,
                                                                   total_num_steps*batch_size),flush=True)
        print('')
        
    
            
            
           
        # populating the lists containing the performance on each batch
        loss_in_steps.append(loss_in_batch)
        accuracy_in_steps.append(accuracy_in_batch)
        r2_heights_in_steps.append(r2_heights_in_batch)

            
    # calculating the mean value of the performance in all the batch
    # the obtained value is considered as the performance in the epoch
    loss_in_epoch=np.mean(np.array(loss_in_steps))
    accuracy_in_epoch=np.mean(np.array(accuracy_in_steps))
    r2_heights_in_epoch=np.mean(np.array(r2_heights_in_steps))
        
    # print the R2 score
    print('r2 in epoch ' +str(epoch)+': '+str(r2_heights_in_epoch))
    print('')
    print('')
    
    # populating the lists containg the performance of each epoch
    loss_all_epochs.append(loss_in_epoch)
    accuracy_all_epochs.append(accuracy_in_epoch)
    r2_heights_all_epochs.append(r2_heights_in_epoch)
        
    # saving the weights of the current epoch
    model.save_weights(weights_folder_path.format(epoch))

# turning the lists containing the performance of each epoch in a numpy array
loss_all_epochs=np.array(loss_all_epochs)
accuracy_all_epochs=np.array(accuracy_all_epochs)
r2_heights_all_epochs=np.array(r2_heights_all_epochs)
    
# saving the arrays containing the performance of each epoch in the folder 'save_performance/training/'
np.save('save_performance/training/loss_all_epochs.npy',loss_all_epochs)
np.save('save_performance/training/accuracy_all_epochs.npy',accuracy_all_epochs)
np.save('save_performance/training/accuracy_all_epochs.npy',r2_heights_all_epochs)
 
    
totaltime = time.time() - before
print('Processing time : {} sec  ({} hours)'.format(totaltime, totaltime/3600))


Training takes action
Training on 2 images in the batch.



Epoch: 0/50 Batch: 0/11   [0/1100]

Training on 2 images in the batch.

Epoch: 0/50 Batch: 1/11   [2/1100]

Training on 2 images in the batch.

Epoch: 0/50 Batch: 2/11   [4/1100]

Training on 2 images in the batch.

